**bold text**# Word 2 Vector + K Nearest Neighbour
## Run 2

In [ ]:
% time # to see wall time 
# Import required libraries
from google.colab import drive
import pickle
import numpy as np
import pandas as pd

# Mount the Google Drive to access the dataset
drive.mount('/content/drive')

# Load the training data from Google Drive
with open('/content/drive/MyDrive/Ablation/data/X_train.pkl', 'rb') as f:
    datax_train = pickle.load(f)

with open('/content/drive/MyDrive/Ablation/data/Y_train.pkl', 'rb') as f:
    datay_train = pickle.load(f)

# Load the validation data from Google Drive
with open('/content/drive/MyDrive/Ablation/data/X_valid.pkl', 'rb') as f:
    datax_valid = pickle.load(f)

with open('/content/drive/MyDrive/Ablation/data/Y_valid.pkl', 'rb') as f:
    datay_valid = pickle.load(f)


# Load the validatiotestn data from Google Drive
with open('//content/drive/MyDrive/Ablation/data//X_test.pkl', 'rb') as f:
    datax_test = pickle.load(f)
    
with open('//content/drive/MyDrive/Ablation/data//Y_test.pkl', 'rb') as f:
    datay_test = pickle.load(f)


#Code was used from Author Code
def prepare_data(seqs, labels, vocabsize, maxlen=None):
    """Create the matrices from the datasets.
    This function pads each sequence to the same length: the length of the
    longest sequence or maxlen. If maxlen is set, all sequences will be cut
    to this maximum length. This function also swaps the axis.
    
    Args:
        seqs (list): A list of sequences.
        labels (list): A list of labels.
        vocabsize (int): The size of the vocabulary.
        maxlen (int): The maximum length of a sequence (optional).
    
    Returns:
        tuple: A tuple containing the following arrays:
            x (np.array): The input matrix of shape (n_samples, maxlen, vocabsize).
            x_mask (np.array): The input mask matrix of shape (n_samples, maxlen).
            y (np.array): The output matrix of shape (n_samples, maxlen).
            lengths (list): A list of sequence lengths.
            eventLengths (list): A list of event sequence lengths.
    """
    
    # Get the lengths of all sequences
    lengths = [len(s) for s in seqs]

    # Concatenate all visits in each sequence to create event sequences
    eventSeq = []
    for seq in seqs:
        t = []
        for visit in seq:
            t.extend(visit)
        eventSeq.append(t)
    eventLengths = [len(s) for s in eventSeq]

    # If maxlen is set, truncate sequences longer than maxlen and discard
    # sequences that are shorter than maxlen
    if maxlen is not None:
        new_seqs = []
        new_lengths = []
        new_labels = []
        for l, s, la in zip(lengths, seqs, labels):
            if l < maxlen:
                new_seqs.append(s)
                new_lengths.append(l)
                new_labels.append(la)
            else:
                new_seqs.append(s[:maxlen])
                new_lengths.append(maxlen)
                new_labels.append(la[:maxlen])
        lengths = new_lengths
        seqs = new_seqs
        labels = new_labels

        # Return None if there are no sequences left after truncation
        if len(lengths) < 1:
            return None, None, None

    n_samples = len(seqs)
    maxlen = np.max(lengths)

    # Initialize the input matrix, the input mask matrix, and the output matrix
    x = np.zeros((n_samples, maxlen, vocabsize)).astype('int64')
    x_mask = np.zeros((n_samples, maxlen)).astype('float64')
    y = np.zeros((n_samples, maxlen)).astype('int64')

    # Fill the input matrix with the one-hot-encoded events
    for idx, s in enumerate(seqs):
        x_mask[idx, :lengths[idx]] = 1
        for j, sj in enumerate(s):
            for tsj in sj:
                x[idx, j, tsj-1] = 1

    # Fill the output matrix with the labels
    for idx, t in enumerate(labels):
        y[idx,:lengths[idx]] = t

    return x, x_mask, y, lengths, eventLengths






CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
Mounted at /content/drive


The code first mounts the Google Drive to access the data. It then loads the training, validation, and testing data from the pickle files.

The prepare_data() function is then defined, which prepares the input data for the CNN model. This function converts the input sequences to a matrix with each row corresponding to a sequence of the same length. Each row contains the one-hot-encoded representation of the events in the sequence. The labels are also converted to a matrix, where each row corresponds to a sequence of labels.

The code then prepares the training, testing, and validation data by calling the prepare_data() function with the required arguments.

Next, the code imports necessary libraries from scikit-learn for classification tasks such as MultiLabelBinarizer, MultiOutputClassifier, LogisticRegression, SVC, GaussianNB, LinearDiscriminantAnalysis. The code also imports the required libraries from Keras for the CNN model such as Sequential, Dense, Flatten, Dropout, Conv1D, MaxPooling1D.

After that, the code reshapes the training, validation, and testing input data to the required shape for the CNN model. Then it creates a CNN model using the Keras library, with two Conv1D layers, followed by a Dropout layer, MaxPooling1D layer, and two Dense layers with softmax activation.

Finally, it extracts the features of the training data using the model_features object and prints them.

In [ ]:
#Creating Feature set with word2Vec words that are saved with different lenghts
#Parameter 1 - Dataset that needs to vectorized
#Parameter 2 - The first number represents vocabulary size that needs to be choosen for samples
#Parameter 3 - The secong number 50 represnets max lenght that needs to used to trim extra length sentecne 
#Code was used from Author Code
# Prepare training data
training = prepare_data(datax_train, datay_train, 100, 50)

# Prepare testing data
testing = prepare_data(datax_test, datay_test, 100, 50)

# Prepare validation data
validation = prepare_data(datax_valid, datay_valid, 100, 50)

# Get training data features
train_X = training[0]
train_X = train_X.reshape(2800, 50*100)

# Get training data labels
train_Y = datay_train

###################

# Get validation data features
valid_X = validation[0]
valid_X = valid_X.reshape(100, 50*100)

# Get validation data labels
valid_Y = datay_valid

####################

# Get testing data features
test_X = testing[0]
test_X = test_X.reshape(100, 50*100)

Get testing data labels
test_Y = datay_test

# Import required libraries
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve,roc_auc_score,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
#The prepare data will return sequences and masks for input along with output
# label for both training and testing

YY=training[2] # extracting training labels for the dataset
YY=np.array(YY) # array conversion


In [ ]:


# Define the estimator and parameters for GridSearchCV
estimator = MultiOutputClassifier(KNeighborsClassifier())
parameters = {
    'estimator__n_neighbors': [3, 5, 7],
    'estimator__weights': ['uniform', 'distance']
}

# Define the GridSearchCV object with f1-score as the scoring metric
clf = GridSearchCV(estimator, parameters, scoring='roc_auc')

# Fit the GridSearchCV object to the training data
clf.fit(train_X, YY)

# Print the best parameters and f1-score
print('Best parameters:', clf.best_params_)
print('Best roc-score:', clf.best_score_)


Best parameters: {'estimator__n_neighbors': 7, 'estimator__weights': 'uniform'}
Best roc-score: 0.538679587092922


This code performs hyperparameter tuning using GridSearchCV for a multi-output K-Nearest Neighbors classifier. Here's a breakdown of what's happening:

from sklearn.model_selection import GridSearchCV: import GridSearchCV from scikit-learn's model selection module.
from sklearn.metrics import f1_score: import the f1_score metric from scikit-learn's metrics module (although it is not used in this code).
from sklearn.neighbors import KNeighborsClassifier: import the KNeighborsClassifier from scikit-learn's neighbors module.
from sklearn.multioutput import MultiOutputClassifier: import the MultiOutputClassifier from scikit-learn's multioutput module.
estimator = MultiOutputClassifier(KNeighborsClassifier()): create a MultiOutputClassifier object with a KNeighborsClassifier estimator. This is the base model to be used in GridSearchCV.
parameters = {'estimator__n_neighbors': [3, 5, 7], 'estimator__weights': ['uniform', 'distance']}: define the hyperparameters to be tuned. In this case, the number of neighbors (n_neighbors) and the type of weighting (weights) to be used in the KNeighborsClassifier.
clf = GridSearchCV(estimator, parameters, scoring='roc_auc'): create a GridSearchCV object with the defined estimator, hyperparameters, and scoring metric (roc_auc). GridSearchCV performs an exhaustive search over the specified hyperparameters and returns the best combination of hyperparameters.
clf.fit(train_X, YY): fit the GridSearchCV object to the training data (train_X and YY).
print('Best parameters:', clf.best_params_): print the best hyperparameters found by GridSearchCV.
print('Best roc-score:', clf.best_score_): print the best roc_auc score obtained by the best hyperparameters.

In [ ]:


# Fit KNeighborsClassifier model with train_X and YY
# Create a MultiOutputClassifier object for multiple outputs
Model = MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=7, weights="uniform")).fit(train_X, YY)

# Use the trained model to predict labels for test_X
y_pred = Model.predict(test_X)


In [ ]:

YY=testing[2]# extracting testing  labels for the dataset
test_YY=np.array(YY)# array conversion

#list to store each sample performance
PR_AUC=[]
ACC=[]
ROC_AUC=[]

for i in range(test_X.shape[0]):
  inter=test_X[i].reshape(1,-1)
  y_score = Model.predict_proba(inter)
  y_pred=Model.predict(inter)
  proba=np.concatenate( y_score, axis=0)
  probabilities=proba[:, 1]
  #ROC_AUC, AU_PRC are define for only for input which has more than 1 classes in 
  #testing data for each sample , so we will calculate the score at that point only 
  #if not we will add zero to list
  if test_YY[i].sum()>0:
    auc_roc=roc_auc_score(test_YY[i],probabilities)
    average_precision = average_precision_score(test_YY[i], probabilities)

  else:
    auc_roc=0 #setting zero if testing lables doesnot have more than 1 classes
    average_precision=0 #setting zero if testing lables doesnot have more than 1 classes
  acc=accuracy_score(test_YY[i],y_pred.reshape(-1,1))
  PR_AUC.append(average_precision)
  ACC.append(acc)
  ROC_AUC.append(auc_roc)
print('ACC score is',sum(ACC)/len(ACC))
print('ROC_AUC score is',sum(ROC_AUC)/len(ROC_AUC))
print('PR_AUC score is',sum(PR_AUC)/len(PR_AUC))
%time

ACC score is 0.7776000000000001
ROC_AUC score is 0.5045758897999183
PR_AUC score is 0.27099363150817973
CPU times: user 10 µs, sys: 37 µs, total: 47 µs
Wall time: 7.87 µs


This code calculates evaluation metrics such as accuracy, PR AUC, and ROC AUC for a given model on the testing data.

The input_testing array contains the features for the testing data and the Model is the trained machine learning model. The loop iterates over each instance of the testing data and makes predictions using the trained model.

The predict_proba method of the trained model is used to obtain the predicted probabilities of the target variable for each instance of the testing data. The probabilities are then used to calculate the PR AUC and ROC AUC scores using the average_precision_score and roc_auc_score functions from the sklearn.metrics module. If the true target variable contains at least one positive label, then the PR AUC and ROC AUC scores are calculated, otherwise, they are set to zero.

The accuracy score is calculated using the accuracy_score function from the sklearn.metrics module. The true target variable is compared with the predicted target variable obtained from the predict method of the trained model.

Finally, the calculated evaluation metrics are stored in the PR_AUC, ACC, and ROC_AUC lists, and the average scores are calculated and printed to the console.

In [ ]:
%whos # to see space

Variable                     Type                     Data/Info
---------------------------------------------------------------
ACC                          list                     n=100
GaussianNB                   ABCMeta                  <class 'sklearn.naive_bayes.GaussianNB'>
GridSearchCV                 ABCMeta                  <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
KNeighborsClassifier         ABCMeta                  <class 'sklearn.neighbors<...>on.KNeighborsClassifier'>
LinearDiscriminantAnalysis   type                     <class 'sklearn.discrimin<...>earDiscriminantAnalysis'>
LogisticRegression           type                     <class 'sklearn.linear_mo<...>stic.LogisticRegression'>
Model                        MultiOutputClassifier    MultiOutputClassifier(est<...>lassifier(n_neighbors=7))
MultiLabelBinarizer          type                     <class 'sklearn.preproces<...>bel.MultiLabelBinarizer'>
MultiOutputClassifier        ABCMeta                  <cl